Guignard Quentin, Stone Tomas, Hanus Maxime

# Question 1 : Explication de type (1-1) pour la Comparaison x > y

## Formulation Mathématique

### 1. Données et Paramètres
- **x, y** : vecteurs de notes des deux candidats pour chaque cours i
- **w** : vecteur des poids de chaque cours
- **contributions(i)** = w[i] × (x[i] - y[i]) : contribution du cours i à la somme pondérée

### 2. Ensembles
- **pros(x,y)** : ensemble des cours i où x[i] > y[i] (contributions > 0)
- **cons(x,y)** : ensemble des cours i où x[i] < y[i] (contributions < 0)
- **neutral(x,y)** : ensemble des cours i où x[i] = y[i] (contributions = 0)

### 3. Formulation du PL

**Variables de décision:**
- Pour chaque paire (P, C) où P \in pros(x,y) et C \in cons(x,y) :
  - $x_{P,C} \in {0, 1}$ : égal à 1 si la paire (P, C) est sélectionnée comme trade-off

**Contraintes:**
1. Chaque cours $C \in \text{cons}(x,y)$ doit être appairé exactement une fois :
   $$\sum_{P \in \text{pros}(x,y)} x_{P,C} = 1 \quad \forall C \in \text{cons}(x,y)$$

2. Chaque cours $P \in \text{pros}(x,y)$ peut être appairé au plus une fois :
   $$\sum_{C \in \text{cons}(x,y)} x_{P,C} \leq 1 \quad \forall P \in \text{pros}(x,y)$$

3. Chaque trade-off sélectionné doit être valide (contribution(P) + contribution(C) > 0) :
   $$(\text{contribution}(P) + \text{contribution}(C)) \cdot x_{P,C} > 0$$
   
   Ou de façon linéaire (si toutes les paires positives) :
   $$\text{contribution}(P) + \text{contribution}(C) > 0 \quad \text{pour chaque paire sélectionnée}$$

**Objectif:**
- Minimiser le nombre de trade-offs : $\min \sum_{P,C} x_{P,C}$
- Ou maximiser pour chercher une explication quelconque : $\max \sum_{P,C} x_{P,C}$

**Certificat de non-existence:**
Si le PL est infaisable (aucune solution), il n'existe pas d'explication (1-1).

In [35]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import pandas as pd 

# Données
courses = ['Anatomie', 'Biologie', 'Chirurgie', 'Diagnostic', 'Epidemiologie', 'Forensic', 'Genetique']
x_notes = np.array([85, 81, 71, 69, 75, 81, 88])
y_notes = np.array([81, 81, 75, 63, 67, 88, 95])
z_notes = np.array([74, 89, 74, 81, 68, 84, 79])
t_notes = np.array([74, 71, 84, 91, 77, 76, 73])
u_notes = np.array([72, 75, 66, 85, 88, 66, 93])
v_notes = np.array([71, 73, 63, 92, 76, 79, 93])
w_notes = np.array([79, 69, 78, 76, 67, 84, 79])
w_prime_notes = np.array([57, 76, 81, 76, 82, 86, 77])
weights = np.array([8, 7, 7, 6, 6, 5, 6])


In [36]:
def find_explanation_1_1(x_values, y_values, weights, features, candidate_x='X', candidate_y='Y'):
    """
    Trouve une explication (1-1) minimale pour x > y
    En (1-1), chaque pro couvre exactement un cons.
    Retourne: (status, explanation)
    """

    # Contributions pondérées
    contributions = weights * (x_values - y_values)
    pros = [i for i in range(len(features)) if contributions[i] > 0]
    cons = [i for i in range(len(features)) if contributions[i] < 0]
    neutral = [i for i in range(len(features)) if contributions[i] == 0]

    print(f'\n=== Analyse {candidate_x} > {candidate_y} ===')
    print(f'pros({candidate_x},{candidate_y}):  ', [features[i] for i in pros])
    print(f'cons({candidate_x},{candidate_y}):  ', [features[i] for i in cons])
    print(f'Contributions: {contributions}')

    if not cons:
        print(f'{candidate_x} domine {candidate_y} sur tous les critères.')
        return 'TRIVIAL', []
    
    if not pros:
        print(f'{candidate_y} domine {candidate_x} sur tous les critères.')
        return 'IMPOSSIBLE', []

    # Trade-offs (1-1) valides: contribution(P)+contribution(C) > 0
    valid_tradeoffs = []
    for p in pros:
        for c in cons:
            s = contributions[p] + contributions[c]
            if s > 0:
                valid_tradeoffs.append((p, c, s))

    print(f'trade-offs valides({candidate_x},{candidate_y}):', [(features[p], features[c], s) for p, c, s in valid_tradeoffs])
        
    # Résolution PL pour une explication (1-1) minimale
    m = gp.Model('explication_1_1')
    x_vars = {}
    for p, c, _ in valid_tradeoffs:
        x_vars[(p,c)] = m.addVar(vtype=GRB.BINARY, name=f'x_{p}_{c}')
    m.update()

    # Chaque cons est couvert exactement une fois
    for c in cons:
        pairs = [(p, cc) for (p, cc, _) in valid_tradeoffs if cc == c]
        m.addConstr(gp.quicksum(x_vars[(p, cc)] for (p, cc) in pairs) == 1)

    # Chaque pro est utilisé au plus une fois
    for p in pros:
        pairs = [(pp, c) for (pp, c, _) in valid_tradeoffs if pp == p]
        m.addConstr(gp.quicksum(x_vars[(pp, c)] for (pp, c) in pairs) <= 1)

    # Objectif: minimiser le nombre de paires
    m.setObjective(gp.quicksum(x_vars.values()), GRB.MINIMIZE)
    m.params.outputflag = 0
    m.optimize()

    if m.status == GRB.OPTIMAL:
        print('\nExplication (1-1) trouvée de longueur', int(m.objVal))
        chosen = []
        for (p, c), var in x_vars.items():
            if var.X > 0.5:
                chosen.append((p, c))
                pc = contributions[p]
                cc = contributions[c]
                print(f"Trade-off : {features[p]} vs. {features[c]}")
                print(f' Contributions : {pc} + ({cc}) = {pc+cc}')
        return 'OPTIMAL', chosen
    else:
        print('Pas de solution (certificat de non-existence).')
        return 'INFEASIBLE', []

In [37]:
result_xy, expl_xy = find_explanation_1_1(x_notes, y_notes, weights, courses, 'x', 'y')


=== Analyse x > y ===
pros(x,y):   ['Anatomie', 'Diagnostic', 'Epidemiologie']
cons(x,y):   ['Chirurgie', 'Forensic', 'Genetique']
Contributions: [ 32   0 -28  36  48 -35 -42]
trade-offs valides(x,y): [('Anatomie', 'Chirurgie', np.int64(4)), ('Diagnostic', 'Chirurgie', np.int64(8)), ('Diagnostic', 'Forensic', np.int64(1)), ('Epidemiologie', 'Chirurgie', np.int64(20)), ('Epidemiologie', 'Forensic', np.int64(13)), ('Epidemiologie', 'Genetique', np.int64(6))]

Explication (1-1) trouvée de longueur 3
Trade-off : Anatomie vs. Chirurgie
 Contributions : 32 + (-28) = 4
Trade-off : Diagnostic vs. Forensic
 Contributions : 36 + (-35) = 1
Trade-off : Epidemiologie vs. Genetique
 Contributions : 48 + (-42) = 6


# Question 2 : Explication de type (1-m) pour la Comparaison x > y

## Formulation Mathématique

### 1. Définition d'un Trade-off (1-m)
Un trade-off (1-m) est une paire (P, {C₁, ..., Cₘ}) où :
- P $\in$ pros(x,y) : un cours où x fait mieux que y
- {C₁, ..., Cₘ} $\subset$ cons(x,y) : m cours où y fait mieux que x
- La somme des contributions est positive : contribution(P) + Σⱼ contribution(Cⱼ) > 0

### 2. Explication (1-m)
Une explication (1-m) de x > y est un ensemble de trade-offs (1-m) disjoints qui couvrent tous les cons(x,y).

### 3. Formulation du Programme Linéaire

**Variables de décision:**
- Pour chaque P $\in$ pros(x,y) et chaque C \in cons(x,y) :
  - $x_{P,C} \in \{0, 1\}$ : égal à 1 si le cours C est associé au cours P dans un trade-off

**Contraintes:**
1. Chaque cours $C \in \text{cons}(x,y)$ doit être appairé exactement une fois :
   $$\sum_{P \in \text{pros}(x,y)} x_{P,C} = 1 \quad \forall C \in \text{cons}(x,y)$$

2. Pour chaque cours $P \in \text{pros}(x,y)$, si P est utilisé, son trade-off doit être valide :
   $$\text{contribution}(P) + \sum_{C \in \text{cons}(x,y)} \text{contribution}(C) \cdot x_{P,C} > 0$$
   
   Version linéarisée avec $y_P \in \{0,1\}$ (indicateur si P est utilisé) :
   $$\text{contribution}(P) \cdot y_P + \sum_{C \in \text{cons}(x,y)} \text{contribution}(C) \cdot x_{P,C} \geq \epsilon \cdot y_P$$
   
   où $\epsilon > 0$ est une petite constante (e.g., 0.001).

3. Lien entre $x_{P,C}$ et $y_P$ :
   $$x_{P,C} \leq y_P \quad \forall P \in \text{pros}(x,y), C \in \text{cons}(x,y)$$
   $$y_P \leq \sum_{C \in \text{cons}(x,y)} x_{P,C} \quad \forall P \in \text{pros}(x,y)$$

**Objectif:**
- Minimiser le nombre de trade-offs : $\min \sum_{P \in \text{pros}(x,y)} y_P$

**Certificat de non-existence:**
Si le PL est infaisable, il n'existe pas d'explication (1-m).

In [38]:
# Implémentation du PL pour explication (1-m)
def find_explanation_1_m(x_values, y_values, weights, features, candidate_x='X', candidate_y='Y'):
    """
    Trouve une explication (1-m) minimale pour x > y
    Retourne: (status, explanation) où status est 'OPTIMAL' ou 'INFEASIBLE'
    """
    # Contributions pondérées
    contributions = weights * (x_values - y_values)
    pros = [i for i in range(len(features)) if contributions[i] > 0]
    cons = [i for i in range(len(features)) if contributions[i] < 0]
    
    print(f'\n=== Analyse {candidate_x} > {candidate_y} ===')
    print(f'pros({candidate_x},{candidate_y}):  ', [features[i] for i in pros])
    print(f'cons({candidate_x},{candidate_y}):  ', [features[i] for i in cons])
    print(f'Contributions: {contributions}')
    
    if not cons:
        print(f'{candidate_x} domine {candidate_y} sur tous les critères.')
        return 'TRIVIAL', []
    
    if not pros:
        print(f'{candidate_y} domine {candidate_x} sur tous les critères.')
        return 'IMPOSSIBLE', []
    
    # Modèle PL
    m = gp.Model('explication_1_m')
    m.params.outputflag = 0
    
    # Variables: x[p,c] = 1 si C est associé à P
    x_vars = {}
    for p in pros:
        for c in cons:
            x_vars[(p,c)] = m.addVar(vtype=GRB.BINARY, name=f'x_{p}_{c}')
    
    # Variables: y[p] = 1 si P est utilisé dans un trade-off
    y_vars = {}
    for p in pros:
        y_vars[p] = m.addVar(vtype=GRB.BINARY, name=f'y_{p}')
    
    m.update()
    
    epsilon = 0.001
    
    # Contrainte 1: Chaque cons est couvert exactement une fois
    for c in cons:
        m.addConstr(gp.quicksum(x_vars[(p,c)] for p in pros) == 1, name=f'cover_cons_{c}')
    
    # Contrainte 2: Validité des trade-offs
    for p in pros:
        sum_contrib = contributions[p] * y_vars[p]
        for c in cons:
            sum_contrib += contributions[c] * x_vars[(p,c)]
        m.addConstr(sum_contrib >= epsilon * y_vars[p], name=f'valid_tradeoff_{p}')
    
    # Contrainte 3: Lien entre x et y
    for p in pros:
        for c in cons:
            m.addConstr(x_vars[(p,c)] <= y_vars[p], name=f'link1_{p}_{c}')
        m.addConstr(y_vars[p] <= gp.quicksum(x_vars[(p,c)] for c in cons), name=f'link2_{p}')
    
    # Objectif: minimiser le nombre de trade-offs
    m.setObjective(gp.quicksum(y_vars.values()), GRB.MINIMIZE)
    m.optimize()
    
    if m.status == GRB.OPTIMAL:
        print(f'\n Explication (1-m) trouvée, longueur = {int(m.objVal)}')
        explanation = []
        for p in pros:
            if y_vars[p].X > 0.5:
                cons_in_tradeoff = [c for c in cons if x_vars[(p,c)].X > 0.5]
                contrib_p = contributions[p]
                contrib_cons = [contributions[c] for c in cons_in_tradeoff]
                total = contrib_p + sum(contrib_cons)
                explanation.append((p, cons_in_tradeoff))
                print(f'  Trade-off: ({features[p]}) vs {[features[c] for c in cons_in_tradeoff]}')
                print(f'    Contributions: {contrib_p:.1f} + {contrib_cons} = {total:.1f}')
        return 'OPTIMAL', explanation
    else:
        print(f'\nPas d\'explication (1-m) (certificat de non-existence)')
        return 'INFEASIBLE', []

# Test avec Xavier vs Yvonne
result, expl = find_explanation_1_m(x_notes, y_notes, weights, courses, 'Xavier', 'Yvonne')


=== Analyse Xavier > Yvonne ===
pros(Xavier,Yvonne):   ['Anatomie', 'Diagnostic', 'Epidemiologie']
cons(Xavier,Yvonne):   ['Chirurgie', 'Forensic', 'Genetique']
Contributions: [ 32   0 -28  36  48 -35 -42]

 Explication (1-m) trouvée, longueur = 3
  Trade-off: (Anatomie) vs ['Chirurgie']
    Contributions: 32.0 + [np.int64(-28)] = 4.0
  Trade-off: (Diagnostic) vs ['Forensic']
    Contributions: 36.0 + [np.int64(-35)] = 1.0
  Trade-off: (Epidemiologie) vs ['Genetique']
    Contributions: 48.0 + [np.int64(-42)] = 6.0


## Démonstration : Absence d'explication (1-m) pour u > v

Nous allons maintenant prouver qu'il n'existe pas d'explication (1-m) pour la comparaison u > v.

In [39]:
# Test de l'explication (1-m) pour u > v
result_uv, expl_uv = find_explanation_1_m(u_notes, v_notes, weights, courses, 'u', 'v')


=== Analyse u > v ===
pros(u,v):   ['Anatomie', 'Biologie', 'Chirurgie', 'Epidemiologie']
cons(u,v):   ['Diagnostic', 'Forensic']
Contributions: [  8  14  21 -42  72 -65   0]

Pas d'explication (1-m) (certificat de non-existence)


## Explication de type (m-1) pour y > z

Un trade-off (m-1) est une paire ({P₁, ..., Pₘ}, C) où :
- {P₁, ..., Pₘ} $\subset$ pros(y,z) : m cours où y fait mieux que z
- C \in cons(y,z) : un cours où z fait mieux que y
- La somme des contributions est positive : Σⱼ contribution(Pⱼ) + contribution(C) > 0

### Formulation du Programme Linéaire pour (m-1)

**Variables:**
- $x_{P,C} \in \{0, 1\}$ : égal à 1 si P est associé au cons C
- $y_C \in \{0, 1\}$ : égal à 1 si C est utilisé dans un trade-off

**Contraintes:**
1. Chaque P \in pros(y,z) est associé à exactement un C :
   $$\sum_{C \in \text{cons}(y,z)} x_{P,C} = 1 \quad \forall P \in \text{pros}(y,z)$$

2. Validité des trade-offs (pour chaque C utilisé) :
   $$\sum_{P \in \text{pros}(y,z)} \text{contribution}(P) \cdot x_{P,C} + \text{contribution}(C) \cdot y_C \geq \epsilon \cdot y_C$$

3. Lien entre x et y :
   $$x_{P,C} \leq y_C \quad \forall P, C$$
   $$y_C \leq \sum_{P \in \text{pros}(y,z)} x_{P,C} \quad \forall C$$

**Objectif:** $\min \sum_C y_C$

In [40]:
# Implémentation du PL pour explication (m-1)
def find_explanation_m_1(x_values, y_values, weights, courses, candidate_x='X', candidate_y='Y'):
    """
    Trouve une explication (m-1) minimale pour x > y
    En (m-1), chaque cons doit être couvert par exactement un trade-off.
    Retourne: (status, explanation)
    """
    # Contributions pondérées
    contributions = weights * (x_values - y_values)
    pros = [i for i in range(len(courses)) if contributions[i] > 0]
    cons = [i for i in range(len(courses)) if contributions[i] < 0]
    
    print(f'\n=== Analyse {candidate_x} > {candidate_y} (type m-1) ===')
    print(f'pros({candidate_x},{candidate_y}):  ', [courses[i] for i in pros], 'contributions:', [contributions[i] for i in pros])
    print(f'cons({candidate_x},{candidate_y}):  ', [courses[i] for i in cons], 'contributions:', [contributions[i] for i in cons])
    
    if not cons:
        print(f'{candidate_x} domine {candidate_y} sur tous les critères.')
        return 'TRIVIAL', []
    
    if not pros:
        print(f'{candidate_y} domine {candidate_x} sur tous les critères.')
        return 'IMPOSSIBLE', []
    
    # Modèle PL
    m = gp.Model('explication_m_1')
    m.params.outputflag = 0
    
    # Variables: x[p,c] = 1 si P est associé à C
    x_vars = {}
    for p in pros:
        for c in cons:
            x_vars[(p,c)] = m.addVar(vtype=GRB.BINARY, name=f'x_{p}_{c}')
    
    # Variables: y[c] = 1 si C est utilisé dans un trade-off (sera forcé à 1)
    y_vars = {}
    for c in cons:
        y_vars[c] = m.addVar(vtype=GRB.BINARY, name=f'y_{c}')
    
    m.update()
    
    epsilon = 0.0
    
    # Contrainte 1: Chaque cons DOIT être couvert (y_c = 1)
    for c in cons:
        m.addConstr(y_vars[c] == 1, name=f'cover_con_{c}')
    
    # Contrainte 2: Chaque pro est utilisé au plus une fois
    for p in pros:
        m.addConstr(gp.quicksum(x_vars[(p,c)] for c in cons) <= 1, name=f'pro_used_once_{p}')
    
    # Contrainte 3: Validité des trade-offs
    # Pour chaque cons, la somme des contributions (cons + ses pros associés) doit être >= 0
    for c in cons:
        sum_contrib = contributions[c] * y_vars[c]
        sum_contrib += gp.quicksum(contributions[p] * x_vars[(p,c)] for p in pros)
        m.addConstr(sum_contrib >= epsilon * y_vars[c], name=f'valid_tradeoff_{c}')
    
    # Contrainte 4: Lien entre x et y (y[c] contrôle quels x peuvent être non-zéro)
    for c in cons:
        for p in pros:
            m.addConstr(x_vars[(p,c)] <= y_vars[c], name=f'link1_{p}_{c}')
        m.addConstr(y_vars[c] <= gp.quicksum(x_vars[(p,c)] for p in pros), name=f'link2_{c}')
    
    # Objectif: minimiser le nombre total de pros utilisés (plutôt que sum(y) qui est constant)
    m.setObjective(gp.quicksum(x_vars.values()), GRB.MINIMIZE)
    m.optimize()
    
    if m.status == GRB.OPTIMAL:
        # Compter le nombre de trade-offs (= nombre de cons couverts = len(cons))
        num_tradeoffs = sum(1 for c in cons if y_vars[c].X > 0.5)
        print(f'\n Explication (m-1) trouvée, longueur = {num_tradeoffs}')
        explanation = []
        for c in cons:
            if y_vars[c].X > 0.5:
                pros_in_tradeoff = [p for p in pros if x_vars[(p,c)].X > 0.5]
                contrib_c = contributions[c]
                contrib_pros = [contributions[p] for p in pros_in_tradeoff]
                total = sum(contrib_pros) + contrib_c
                explanation.append((pros_in_tradeoff, c))
                print(f'  Trade-off: {[courses[p] for p in pros_in_tradeoff]} vs ({courses[c]})')
                print(f'    Contributions: {contrib_pros} + {contrib_c:.1f} = {total:.1f}')
        return 'OPTIMAL', explanation
    else:
        print(f'\n  Pas d\'explication (m-1) (certificat de non-existence)')
        return 'INFEASIBLE', []

# Test de l'explication (m-1) pour y > z
result_yz, expl_yz = find_explanation_m_1(y_notes, z_notes, weights, courses, 'y', 'z')


=== Analyse y > z (type m-1) ===
pros(y,z):   ['Anatomie', 'Chirurgie', 'Forensic', 'Genetique'] contributions: [np.int64(56), np.int64(7), np.int64(20), np.int64(96)]
cons(y,z):   ['Biologie', 'Diagnostic', 'Epidemiologie'] contributions: [np.int64(-56), np.int64(-108), np.int64(-6)]

 Explication (m-1) trouvée, longueur = 3
  Trade-off: ['Anatomie'] vs (Biologie)
    Contributions: [np.int64(56)] + -56.0 = 0.0
  Trade-off: ['Forensic', 'Genetique'] vs (Diagnostic)
    Contributions: [np.int64(20), np.int64(96)] + -108.0 = 8.0
  Trade-off: ['Chirurgie'] vs (Epidemiologie)
    Contributions: [np.int64(7)] + -6.0 = 1.0


# Question 3 : Explications Combinées (m-1) et (1-m)

## Formulation Mathématique

### 1. Définitions

Une **explication combinée** de x > y est un ensemble E = {(P₁^(m₁), C₁), ..., (Pₖ^(mₖ), Cₖ), (P'₁, C'₁^(n₁)), ..., (P'ₗ, C'ₗ^(nₗ))} où :
- Les k premiers sont des trade-offs de type (m-1) : chaque (Pᵢ^(mᵢ), Cᵢ) a mᵢ pros et 1 cons
- Les ℓ derniers sont des trade-offs de type (1-m) : chaque (P'ⱼ, C'ⱼ^(nⱼ)) a 1 pro et nⱼ cons
- Tous les cons(x,y) sont couverts exactement une fois
- Chaque trade-off est valide (somme des contributions > 0)

### 2. Formulation ILP Combinée

**Variables:**
- $x_{p,c} \in \{0, 1\}$ : pro p est associé au cons c dans un trade-off
- $z_c \in \{0, 1\}$ : cons c est utilisé dans un trade-off (type m-1 si z_c=1)
- $w_p \in \{0, 1\}$ : pro p est utilisé dans un trade-off (type 1-m si w_p=1)
- $y_c \in \{0, 1\}$ : cons c est couvert (dans n'importe quel trade-off)

**Contraintes:**

1. **Couverture des cons** (exactement une fois) :
   $$\sum_{p \in pros} x_{p,c} = 1 \quad \forall c \in cons$$

2. **Chaque pro au plus une fois** :
   $$\sum_{c \in cons} x_{p,c} \leq 1 \quad \forall p \in pros$$

3. **Validité des trade-offs** :
   $$\sum_{p \in pros} \omega_p \cdot x_{p,c} + \omega_c \geq 0 \quad \forall c \in cons$$

4. **Activation des indicateurs** :
   - Si c est couvert : $y_c = 1$
   - Si c est en (m-1) : $z_c \leq \sum_p x_{p,c}$
   - Si c est en (1-m) : $w_p \leq \sum_c x_{p,c}$

**Objectif:**
- Minimiser la longueur totale : $\min \sum_c y_c$

### 3. Analyse de z > t

Pour z > t, nous allons montrer :
1. Qu'il n'existe pas d'explication (1-m) seule
2. Qu'il n'existe pas d'explication (m-1) seule
3. Qu'une explication combinée existe

In [41]:
# Vérification pour z > t : pas de (1-m) ni (m-1) seule
print('\n' + '='*80)
print('QUESTION 3 : Analyse de z > t')
print('='*80)

print('\n--- Test (1-m) pour z > t ---')
result_1m, expl_1m = find_explanation_1_m(z_notes, t_notes, weights, courses, 'z', 't')

print('\n--- Test (m-1) pour z > t ---')
result_m1, expl_m1 = find_explanation_m_1(z_notes, t_notes, weights, courses, 'z', 't')


QUESTION 3 : Analyse de z > t

--- Test (1-m) pour z > t ---

=== Analyse z > t ===
pros(z,t):   ['Biologie', 'Forensic', 'Genetique']
cons(z,t):   ['Chirurgie', 'Diagnostic', 'Epidemiologie']
Contributions: [  0 126 -70 -60 -54  40  36]

Pas d'explication (1-m) (certificat de non-existence)

--- Test (m-1) pour z > t ---

=== Analyse z > t (type m-1) ===
pros(z,t):   ['Biologie', 'Forensic', 'Genetique'] contributions: [np.int64(126), np.int64(40), np.int64(36)]
cons(z,t):   ['Chirurgie', 'Diagnostic', 'Epidemiologie'] contributions: [np.int64(-70), np.int64(-60), np.int64(-54)]

  Pas d'explication (m-1) (certificat de non-existence)


In [42]:
# Implémentation du PL pour explication mixte (m-1) + (1-m)
def find_explanation_mixed(x_values, y_values, weights, courses, candidate_x='X', candidate_y='Y'):
    """
    Trouve une explication mixte combinant (m-1) et (1-m) pour x > y
    - (m-1): plusieurs pros couvrent 1 cons
    - (1-m): 1 pro couvre plusieurs cons
    Retourne: (status, explanation)
    """
    x_values = np.array(x_values, dtype=float)
    y_values = np.array(y_values, dtype=float)
    weights = np.array(weights, dtype=float)

    omega = weights * (x_values - y_values)
    pros = [i for i in range(len(courses)) if omega[i] > 0]
    cons = [i for i in range(len(courses)) if omega[i] < 0]

    print(f'\n=== Analyse {candidate_x} > {candidate_y} (type mixte) ===')
    print(f'pros({candidate_x},{candidate_y}):  ', [courses[i] for i in pros], 'contributions:', [omega[i] for i in pros])
    print(f'cons({candidate_x},{candidate_y}):  ', [courses[i] for i in cons], 'contributions:', [omega[i] for i in cons])

    if not cons:
        print(f'{candidate_x} domine {candidate_y} sur tous les critères.')
        return 'TRIVIAL', []
    if not pros:
        print(f'{candidate_y} domine {candidate_x} sur tous les critères.')
        return 'IMPOSSIBLE', []

    M = float(np.sum(np.abs(omega)) + 1.0)
    epsilon = 0.0

    m = gp.Model('explication_mixte')
    m.Params.OutputFlag = 0

    # Variables (m-1): x_pc[p,c] = 1 si pro p est associé au cons c dans un trade-off (m-1)
    x_pc = {(p,c): m.addVar(vtype=GRB.BINARY, name=f'x_{p}_{c}') for p in pros for c in cons}
    # y_c[c] = 1 si cons c est couvert par un trade-off (m-1)
    y_c = {c: m.addVar(vtype=GRB.BINARY, name=f'y_{c}') for c in cons}

    # Variables (1-m): z_pc[p,c] = 1 si cons c est associé au pro p dans un trade-off (1-m)
    z_pc = {(p,c): m.addVar(vtype=GRB.BINARY, name=f'z_{p}_{c}') for p in pros for c in cons}
    # u_p[p] = 1 si pro p est utilisé dans un trade-off (1-m)
    u_p = {p: m.addVar(vtype=GRB.BINARY, name=f'u_{p}') for p in pros}

    m.update()

    # Contrainte 1: Chaque cons couvert exactement une fois (soit par m-1 soit par 1-m)
    for c in cons:
        m.addConstr(y_c[c] + gp.quicksum(z_pc[(p,c)] for p in pros) == 1, name=f'cover_{c}')

    # Contrainte 2: Cohérence et validité des trade-offs (m-1)
    for c in cons:
        for p in pros:
            m.addConstr(x_pc[(p,c)] <= y_c[c], name=f'link_m1_{p}_{c}')
        m.addConstr(gp.quicksum(x_pc[(p,c)] for p in pros) >= y_c[c], name=f'nonempty_m1_{c}')
        m.addConstr(
            omega[c] + gp.quicksum(omega[p] * x_pc[(p,c)] for p in pros) >= epsilon - M*(1 - y_c[c]),
            name=f'valid_m1_{c}'
        )

    # Contrainte 3: Cohérence et validité des trade-offs (1-m)
    for p in pros:
        for c in cons:
            m.addConstr(z_pc[(p,c)] <= u_p[p], name=f'link_1m_{p}_{c}')
        m.addConstr(gp.quicksum(z_pc[(p,c)] for c in cons) >= u_p[p], name=f'nonempty_1m_{p}')
        m.addConstr(
            omega[p] + gp.quicksum(omega[c] * z_pc[(p,c)] for c in cons) >= epsilon - M*(1 - u_p[p]),
            name=f'valid_1m_{p}'
        )

    # Contrainte 4: Pros disjoints (utilisé au plus une fois, pas dans les deux modes)
    for p in pros:
        m.addConstr(gp.quicksum(x_pc[(p,c)] for c in cons) <= 1, name=f'pro_once_m1_{p}')
        m.addConstr(gp.quicksum(x_pc[(p,c)] for c in cons) + u_p[p] <= 1, name=f'pro_disjoint_{p}')

    # Objectif: minimiser le nombre de trade-offs
    m.setObjective(gp.quicksum(y_c[c] for c in cons) + gp.quicksum(u_p[p] for p in pros), GRB.MINIMIZE)
    m.optimize()

    if m.Status != GRB.OPTIMAL:
        print(f'\nPas d\'explication mixte (certificat de non-existence)')
        return 'INFEASIBLE', []

    # Collecter les trade-offs
    explanation = []
    
    # Trade-offs (m-1)
    for c in cons:
        if y_c[c].X > 0.5:
            P = [p for p in pros if x_pc[(p,c)].X > 0.5]
            val = float(omega[c] + sum(omega[p] for p in P))
            explanation.append(('m-1', P, [c], val))

    # Trade-offs (1-m)
    for p in pros:
        if u_p[p].X > 0.5:
            Cset = [c for c in cons if z_pc[(p,c)].X > 0.5]
            val = float(omega[p] + sum(omega[c] for c in Cset))
            explanation.append(('1-m', [p], Cset, val))

    print(f'\n Explication mixte trouvée, longueur = {len(explanation)}')
    for typ, P, Cset, val in explanation:
        if typ == 'm-1':
            print(f'  Trade-off (m-1): {[courses[i] for i in P]} vs ({courses[Cset[0]]})')
            print(f'    Contributions: {[omega[i] for i in P]} + {omega[Cset[0]]:.1f} = {val:.1f}')
        else:
            print(f'  Trade-off (1-m): ({courses[P[0]]}) vs {[courses[i] for i in Cset]}')
            print(f'    Contributions: {omega[P[0]]:.1f} + {[omega[i] for i in Cset]} = {val:.1f}')

    return 'OPTIMAL', explanation

# Test de l'explication mixte pour z > t
print('\n' + '='*80)
print('TEST EXPLICATION MIXTE pour z > t')
print('='*80)

result_mixed, expl_mixed = find_explanation_mixed(z_notes, t_notes, weights, courses, 'z', 't')


TEST EXPLICATION MIXTE pour z > t

=== Analyse z > t (type mixte) ===
pros(z,t):   ['Biologie', 'Forensic', 'Genetique'] contributions: [np.float64(126.0), np.float64(40.0), np.float64(36.0)]
cons(z,t):   ['Chirurgie', 'Diagnostic', 'Epidemiologie'] contributions: [np.float64(-70.0), np.float64(-60.0), np.float64(-54.0)]

 Explication mixte trouvée, longueur = 2
  Trade-off (m-1): ['Forensic', 'Genetique'] vs (Chirurgie)
    Contributions: [np.float64(40.0), np.float64(36.0)] + -70.0 = 6.0
  Trade-off (1-m): (Biologie) vs ['Diagnostic', 'Epidemiologie']
    Contributions: 126.0 + [np.float64(-60.0), np.float64(-54.0)] = 12.0


# Question 4 : Explication de type (1-m) OU (m-1)

## Formulation Mathématique

On cherche une explication qui peut utiliser **soit** des trade-offs (1-m) **soit** des trade-offs (m-1) (mais pas les deux en même temps, contrairement à l'explication mixte).

### Approche

1. Tester d'abord s'il existe une explication (1-m)
2. Si non, tester s'il existe une explication (m-1)
3. Retourner la première explication trouvée, ou un certificat de non-existence

### Données des nouveaux candidats

| Candidat | A | B | C | D | E | F | G |
|----------|---|---|---|---|---|---|---|
| a1 | 89 | 74 | 81 | 68 | 84 | 79 | 77 |
| a2 | 71 | 84 | 91 | 79 | 78 | 73.5 | 77 |

In [43]:
# Définition des données pour a1 et a2
a1_notes = np.array([89, 74, 81, 68, 84, 79, 77])
a2_notes = np.array([71, 84, 91, 79, 78, 73.5, 77])

# Vérification que a1 > a2 (somme pondérée)
score_a1 = np.dot(weights, a1_notes)
score_a2 = np.dot(weights, a2_notes)
print(f'Score pondéré a1 = {score_a1}')
print(f'Score pondéré a2 = {score_a2}')

Score pondéré a1 = 3566
Score pondéré a2 = 3564.5


In [44]:
# Application à a1 > a2
status_1_m, expl_1_m = find_explanation_1_m(a1_notes, a2_notes, weights, courses, 'a1', 'a2')
status_m_1, expl_m_1 = find_explanation_m_1(a1_notes, a2_notes, weights, courses, 'a1', 'a2')

print(f'\n{"="*80}')
print(f'RÉSULTAT FINAL')
print(f'{"="*80}')

if status_1_m == 'OPTIMAL':
    print(f'Explication trouvée de type (1-m) pour a1 > a2')
elif status_m_1 == 'OPTIMAL':
    print(f'Explication trouvée de type (m-1) pour a1 > a2')
else:
    print(f'Pas d\'explication de type (1-m) ni (m-1) pour a1 > a2')


=== Analyse a1 > a2 ===
pros(a1,a2):   ['Anatomie', 'Epidemiologie', 'Forensic']
cons(a1,a2):   ['Biologie', 'Chirurgie', 'Diagnostic']
Contributions: [144.  -70.  -70.  -66.   36.   27.5   0. ]

Pas d'explication (1-m) (certificat de non-existence)

=== Analyse a1 > a2 (type m-1) ===
pros(a1,a2):   ['Anatomie', 'Epidemiologie', 'Forensic'] contributions: [np.float64(144.0), np.float64(36.0), np.float64(27.5)]
cons(a1,a2):   ['Biologie', 'Chirurgie', 'Diagnostic'] contributions: [np.float64(-70.0), np.float64(-70.0), np.float64(-66.0)]

  Pas d'explication (m-1) (certificat de non-existence)

RÉSULTAT FINAL
Pas d'explication de type (1-m) ni (m-1) pour a1 > a2


## Analyse du Résultat

**Conclusion pour a1 > a2 :**

Il n'existe **pas** d'explication de type (1-m) seule, ni de type (m-1) seule pour expliquer à a2 qu'il est moins bien classé que a1.

### Pourquoi ?

Analysons les contributions :
- **pros(a1,a2)** : Anatomie (+144), Epidémiologie (+36), Forensic (+27.5) → Total = 207.5
- **cons(a1,a2)** : Biologie (-70), Chirurgie (-70), Diagnostic (-66) → Total = -206

La différence est très faible (1.5 points), ce qui rend difficile la construction de trade-offs valides.

- **Pour (1-m)** : Aucun pro seul ne peut compenser plusieurs cons
  - Anatomie (144) > Biologie + Chirurgie (140) mais il faudrait aussi couvrir Diagnostic
  
- **Pour (m-1)** : Aucun cons seul ne peut être compensé par un sous-ensemble de pros tout en laissant assez pour les autres cons

### Solution

Une explication **mixte** combinant (1-m) et (m-1) pourrait exister. Testons avec la fonction de la Question 3 :

In [45]:
# Test avec l'explication mixte (Question 3) pour a1 > a2
print("Test de l'explication mixte pour a1 > a2:")
result_a1a2_mixed, expl_a1a2_mixed = find_explanation_mixed(a1_notes, a2_notes, weights, courses, 'a1', 'a2')

Test de l'explication mixte pour a1 > a2:

=== Analyse a1 > a2 (type mixte) ===
pros(a1,a2):   ['Anatomie', 'Epidemiologie', 'Forensic'] contributions: [np.float64(144.0), np.float64(36.0), np.float64(27.5)]
cons(a1,a2):   ['Biologie', 'Chirurgie', 'Diagnostic'] contributions: [np.float64(-70.0), np.float64(-70.0), np.float64(-66.0)]

Pas d'explication mixte (certificat de non-existence)


## Conclusion Finale pour la Question 4

### Résultat : **Aucune explication possible**

Pour la comparaison **a1 > a2**, il n'existe :
- Pas d'explication (1-1)
- Pas d'explication (1-m)
- Pas d'explication (m-1)
- Pas d'explication mixte (1-m) + (m-1)

### Explication Mathématique

| Critère | a1 | a2 | Contribution pondérée |
|---------|----|----|----------------------|
| A (poids 8) | 89 | 71 | +144 |
| B (poids 7) | 74 | 84 | -70 |
| C (poids 7) | 81 | 91 | -70 |
| D (poids 6) | 68 | 79 | -66 |
| E (poids 6) | 84 | 78 | +36 |
| F (poids 5) | 79 | 73.5 | +27.5 |
| G (poids 6) | 77 | 77 | 0 |
| **Total** | | | **+1.5** |

La différence totale n'est que de **1.5 points**, ce qui est extrêmement faible.

### Pourquoi aucun trade-off n'est valide ?

Pour qu'un trade-off soit valide, il faut que la somme des contributions soit **strictement positive**.

- Le plus gros pro (Anatomie: +144) ne suffit pas à couvrir deux cons : 144 - 70 - 70 = +4, mais alors Diagnostic (-66) reste non couvert
- Si Anatomie couvre B et C, il reste D (-66) à couvrir par E (+36) et F (+27.5) = 63.5, ce qui est insuffisant

**Le certificat de non-existence prouve qu'il est impossible de construire une explication simple pour justifier a1 > a2.**


On ne peut donc pas expliquer simplement à a2 pourquoi il est moins bien classé que a1. La préférence existe (score 3566 vs 3564.5), mais elle est trop marginale pour être justifiée par des trade-offs compensatoires.

# Dataset RATP

### Importation du dataset et des poids associés à chaque variable.

In [46]:
ratp_data = pd.read_excel("RATP.xlsx", skiprows=1, skipfooter=2)
ratp_data.drop(columns=["Unnamed: 0"], inplace=True)
ratp_data.rename(columns={f"Unnamed: {i}": ratp_data.iloc[0, :].values[i-1] for i in range(1, ratp_data.shape[1]+1)}, inplace=True)
ratp_data.drop(index=0, inplace=True)
ratp_data.reset_index(drop=True, inplace=True)

weights = pd.read_excel("RATP.xlsx")
weights = weights.iloc[-1, 1:].values[1:]

In [47]:
ratp_data

,Metro station,peak-entering-passengers/h,peak-passing-passengers/h,off-peak-entering-passengers/h,off-peak-passing-passengers/h,"strategic priority [0,10]","Station degradation level ([0,20] scale)","connectivity index [0,100]"
0,Odéon (Ligne 4),85000,8100,35500,3450,75,16.2,88
1,Place d'Italie (Lign 6),81000,8100,37500,3150,67,17.6,95
2,Jussieu (Ligne 7),74000,8900,37000,4050,68,16.8,79
3,Nation (Ligne 9),74000,7100,42000,4550,77,15.2,73
4,La Motte Picquet-Grenelle (Ligne 10),72000,7500,33000,4250,88,13.2,93
5,Porte d'Orléans (Ligne 4),71000,7300,31500,4600,76,15.8,93
6,Daumenil (Ligne 6),79000,6900,39000,3800,67,16.8,79
7,Vaugirard (Ligne 12),57000,7600,40500,3800,82,17.2,77
8,Oberkampf (Ligne 9),84000,7900,34000,3300,74,15.8,85
9,Reuilly-Diderot (Ligne 1),72000,8700,36000,4000,66,16.6,78


### Calcul des scores en fonction des poids

On ajoute 2 colonnes au dataset : 
- Le score de priorité obtenu par la somme pondéré des valeurs des différentes variables pour chaque station.
- Le rang de priorité de chaque station; on trie dans l'ordre décroissant les stations par rapport à leur score.  

In [48]:
ratp_data['Score'] = ratp_data.iloc[:, 1:].values @ weights
ratp_data.sort_values(by='Score', ascending=False, inplace=True)
ratp_data['Rank'] = np.arange(1, ratp_data.shape[0]+1)
ratp_data.reset_index(drop=True, inplace=True)
ratp_data

,Metro station,peak-entering-passengers/h,peak-passing-passengers/h,off-peak-entering-passengers/h,off-peak-passing-passengers/h,"strategic priority [0,10]","Station degradation level ([0,20] scale)","connectivity index [0,100]",Score,Rank
0,Odéon (Ligne 4),85000,8100,35500,3450,75,16.2,88,9515.747608,1
1,Place d'Italie (Lign 6),81000,8100,37500,3150,67,17.6,95,9486.187251,2
2,Jussieu (Ligne 7),74000,8900,37000,4050,68,16.8,79,9462.001505,3
3,Nation (Ligne 9),74000,7100,42000,4550,77,15.2,73,9413.630012,4
4,La Motte Picquet-Grenelle (Ligne 10),72000,7500,33000,4250,88,13.2,93,9376.007739,5
5,Porte d'Orléans (Ligne 4),71000,7300,31500,4600,76,15.8,93,9354.509298,6
6,Reuilly-Diderot (Ligne 1),72000,8700,36000,4000,66,16.6,78,9265.828227,7
7,Oberkampf (Ligne 9),84000,7900,34000,3300,74,15.8,85,9260.453617,8
8,Daumenil (Ligne 6),79000,6900,39000,3800,67,16.8,79,9171.772546,9
9,Vaugirard (Ligne 12),57000,7600,40500,3800,82,17.2,77,9123.401053,10


## Justification du top1

On montre Odéon (Ligne 4) > Place d'Italie (Ligne 6)

In [49]:
odeon = ratp_data.iloc[0].values.flatten()[1:-2].astype(float) 
italie = ratp_data.iloc[1].values.flatten()[1:-2].astype(float)

pros = [i for i in range(len(odeon)) if odeon[i] > italie[i]]
cons = [i for i in range(len(odeon)) if odeon[i] < italie[i]]
neutral = [i for i in range(len(odeon)) if odeon[i] == italie[i]]

print(f"{len(pros)} pros:", [ratp_data.columns[i+1] for i in pros])
print(f"{len(cons)} cons:", [ratp_data.columns[i+1] for i in cons])
print(f"{len(neutral)} neutral:", [ratp_data.columns[i+1] for i in neutral])

3 pros: ['peak-entering-passengers/h', 'off-peak-passing-passengers/h', 'strategic priority [0,10]']
3 cons: ['off-peak-entering-passengers/h', 'Station degradation level ([0,20]  scale)', 'connectivity index [0,100]']
1 neutral: ['peak-passing-passengers/h']


On a 3 pros et 3 cons. L'explication 1-1 paraît être la plus adaptée dans un premier temps. 

In [50]:
result_odeon_italie, expl_odeon_italie = find_explanation_1_1(odeon, italie, weights, ratp_data.columns[1:-2], 'Odeon', 'Place d\'Italie')


=== Analyse Odeon > Place d'Italie ===
pros(Odeon,Place d'Italie):   ['peak-entering-passengers/h', 'off-peak-passing-passengers/h', 'strategic priority [0,10]']
cons(Odeon,Place d'Italie):   ['off-peak-entering-passengers/h', 'Station degradation level ([0,20]  scale)', 'connectivity index [0,100]']
Contributions: [85.99376545200472 0.0 -75.24454477050415 96.7429861335053
 128.9906481780071 -94.0556809631303 -112.86681715575621]
trade-offs valides(Odeon,Place d'Italie): [('peak-entering-passengers/h', 'off-peak-entering-passengers/h', 10.749220681500574), ('off-peak-passing-passengers/h', 'off-peak-entering-passengers/h', 21.49844136300115), ('off-peak-passing-passengers/h', 'Station degradation level ([0,20]  scale)', 2.687305170374998), ('strategic priority [0,10]', 'off-peak-entering-passengers/h', 53.74610340750294), ('strategic priority [0,10]', 'Station degradation level ([0,20]  scale)', 34.93496721487679), ('strategic priority [0,10]', 'connectivity index [0,100]', 16.1238310

## Justification du top3

Jussieu est la station ayant le 3ème score de priorité le plus élevé. On va expliquer ce score par rapport à la station top2 (i.e Place d'Italie > Jussieu) et à la station top4 (i.e Jussieu > Nation).

### Justification Place d'Italie > Jussieu 

In [52]:
italie = ratp_data.iloc[1].values.flatten()[1:-2].astype(float)
jussieu = ratp_data.iloc[2].values.flatten()[1:-2].astype(float) 

pros = [i for i in range(len(italie)) if italie[i] > jussieu[i]]
cons = [i for i in range(len(italie)) if italie[i] < jussieu[i]]
neutral = [i for i in range(len(italie)) if italie[i] == jussieu[i]]

print(f"{len(pros)} pros:", [ratp_data.columns[i+1] for i in pros])
print(f"{len(cons)} cons:", [ratp_data.columns[i+1] for i in cons])
print(f"{len(neutral)} neutral:", [ratp_data.columns[i+1] for i in neutral])

4 pros: ['peak-entering-passengers/h', 'off-peak-entering-passengers/h', 'Station degradation level ([0,20]  scale)', 'connectivity index [0,100]']
3 cons: ['peak-passing-passengers/h', 'off-peak-passing-passengers/h', 'strategic priority [0,10]']
0 neutral: []


On a plus de pros que de cons. On peut essayer une explication (1-1) et si cela ne marche pas une explication (m-1) devrait marcher. 

In [53]:
result_italie_jussieu, expl_italie_jussieu = find_explanation_1_1(italie, jussieu, weights, ratp_data.columns[1:-2], 'Place d\'Italie', 'Jussieu')


=== Analyse Place d'Italie > Jussieu ===
pros(Place d'Italie,Jussieu):   ['peak-entering-passengers/h', 'off-peak-entering-passengers/h', 'Station degradation level ([0,20]  scale)', 'connectivity index [0,100]']
cons(Place d'Italie,Jussieu):   ['peak-passing-passengers/h', 'off-peak-passing-passengers/h', 'strategic priority [0,10]']
Contributions: [150.48908954100827 -150.48908954100827 18.811136192626037
 -290.2289584005159 -16.123831022250886 53.746103407503 257.9812963560142]
trade-offs valides(Place d'Italie,Jussieu): [('peak-entering-passengers/h', 'strategic priority [0,10]', 134.36525851875737), ('off-peak-entering-passengers/h', 'strategic priority [0,10]', 2.6873051703751507), ('Station degradation level ([0,20]  scale)', 'strategic priority [0,10]', 37.62227238525212), ('connectivity index [0,100]', 'peak-passing-passengers/h', 107.49220681500591), ('connectivity index [0,100]', 'strategic priority [0,10]', 241.8574653337633)]
Pas de solution (certificat de non-existence).

L'explication (1-1) ne marche pas, on essaie donc d'expliquer avec une méthode (m-1) pour compenser un con par plusieurs pros. 

In [54]:
result_italie_jussieu, expl_italie_jussieu = find_explanation_m_1(italie, jussieu, weights, ratp_data.columns[1:-2], 'Place d\'Italie', 'Jussieu')


=== Analyse Place d'Italie > Jussieu (type m-1) ===
pros(Place d'Italie,Jussieu):   ['peak-entering-passengers/h', 'off-peak-entering-passengers/h', 'Station degradation level ([0,20]  scale)', 'connectivity index [0,100]'] contributions: [150.48908954100827, 18.811136192626037, 53.746103407503, 257.9812963560142]
cons(Place d'Italie,Jussieu):   ['peak-passing-passengers/h', 'off-peak-passing-passengers/h', 'strategic priority [0,10]'] contributions: [-150.48908954100827, -290.2289584005159, -16.123831022250886]

 Explication (m-1) trouvée, longueur = 3
  Trade-off: ['peak-entering-passengers/h'] vs (peak-passing-passengers/h)
    Contributions: [150.48908954100827] + -150.5 = 0.0
  Trade-off: ['Station degradation level ([0,20]  scale)', 'connectivity index [0,100]'] vs (off-peak-passing-passengers/h)
    Contributions: [53.746103407503, 257.9812963560142] + -290.2 = 21.5
  Trade-off: ['off-peak-entering-passengers/h'] vs (strategic priority [0,10])
    Contributions: [18.81113619262

### Justification Jussieu > Nation

In [56]:
jussieu = ratp_data.iloc[2].values.flatten()[1:-2].astype(float)
nation = ratp_data.iloc[3].values.flatten()[1:-2].astype(float)

pros = [i for i in range(len(jussieu)) if jussieu[i] > nation[i]]
cons = [i for i in range(len(jussieu)) if jussieu[i] < nation[i]]
neutral = [i for i in range(len(jussieu)) if jussieu[i] == nation[i]]

print(f"{len(pros)} pros:", [ratp_data.columns[i+1] for i in pros])
print(f"{len(cons)} cons:", [ratp_data.columns[i+1] for i in cons])
print(f"{len(neutral)} neutral:", [ratp_data.columns[i+1] for i in neutral])

3 pros: ['peak-passing-passengers/h', 'Station degradation level ([0,20]  scale)', 'connectivity index [0,100]']
3 cons: ['off-peak-entering-passengers/h', 'off-peak-passing-passengers/h', 'strategic priority [0,10]']
1 neutral: ['peak-entering-passengers/h']


On a 3 pros et 3 cons, une explication (1-1) semble logique. 

In [57]:
result_jussieu_nation, expl_jussieu_nation = find_explanation_1_1(jussieu, nation, weights, ratp_data.columns[1:-2], 'Jussieu', 'Nation')


=== Analyse Jussieu > Nation ===
pros(Jussieu,Nation):   ['peak-passing-passengers/h', 'Station degradation level ([0,20]  scale)', 'connectivity index [0,100]']
cons(Jussieu,Nation):   ['off-peak-entering-passengers/h', 'off-peak-passing-passengers/h', 'strategic priority [0,10]']
Contributions: [0.0 338.6004514672686 -188.11136192626037 -161.23831022250883
 -145.114479200258 107.49220681500587 96.74298613350533]
trade-offs valides(Jussieu,Nation): [('peak-passing-passengers/h', 'off-peak-entering-passengers/h', 150.48908954100824), ('peak-passing-passengers/h', 'off-peak-passing-passengers/h', 177.36214124475978), ('peak-passing-passengers/h', 'strategic priority [0,10]', 193.48597226701062)]
Pas de solution (certificat de non-existence).


L'explication (1-1) ne marche pas. On voit une contribution assez grande par rapport aux autres cons mais que les autres pros sont tous plus petits que les cons. On tente donc une explication mixte. 

In [58]:
result_jussieu_nation, expl_jussieu_nation = find_explanation_mixed(jussieu, nation, weights, ratp_data.columns[1:-2], 'Jussieu', 'Nation')


=== Analyse Jussieu > Nation (type mixte) ===
pros(Jussieu,Nation):   ['peak-passing-passengers/h', 'Station degradation level ([0,20]  scale)', 'connectivity index [0,100]'] contributions: [np.float64(338.6004514672686), np.float64(107.49220681500587), np.float64(96.74298613350533)]
cons(Jussieu,Nation):   ['off-peak-entering-passengers/h', 'off-peak-passing-passengers/h', 'strategic priority [0,10]'] contributions: [np.float64(-188.11136192626037), np.float64(-161.23831022250883), np.float64(-145.114479200258)]

 Explication mixte trouvée, longueur = 2
  Trade-off (m-1): ['Station degradation level ([0,20]  scale)', 'connectivity index [0,100]'] vs (off-peak-entering-passengers/h)
    Contributions: [np.float64(107.49220681500587), np.float64(96.74298613350533)] + -188.1 = 16.1
  Trade-off (1-m): (peak-passing-passengers/h) vs ['off-peak-passing-passengers/h', 'strategic priority [0,10]']
    Contributions: 338.6 + [np.float64(-161.23831022250883), np.float64(-145.114479200258)] = 3

On voit que ``peak-passing-passengers/h`` réussi à absorber les 2 cons ``off-peak-passing-passengers/h`` et ``strategic priority [0,10]`` et que les 2 pros ``Station degradation level ([0,20]  scale)`` et  ``connectivity index [0,100]`` compensent ``off-peak-entering-passengers/h`` 

## Justification top5

La Motte Picquet-Grenelle est la station ayant le 5ème score de priorité le plus élevé. On va expliquer ce score par rapport à la station top4 (i.e Nation > La Motte Picquet-Grenelle) et à la station top6 (i.e La Motte Picquet-Grenelle > Porte d'Orléans).

### On montre Nation > La Motte Picquet-Grenelle

In [59]:
nation = ratp_data.iloc[3].values.flatten()[1:-2].astype(float)
lamotte = ratp_data.iloc[4].values.flatten()[1:-2].astype(float)

pros = [i for i in range(len(nation)) if nation[i] > lamotte[i]]
cons = [i for i in range(len(nation)) if nation[i] < lamotte[i]]
neutral = [i for i in range(len(nation)) if nation[i] == lamotte[i]]

print(f"{len(pros)} pros:", [ratp_data.columns[i+1] for i in pros])
print(f"{len(cons)} cons:", [ratp_data.columns[i+1] for i in cons])
print(f"{len(neutral)} neutral:", [ratp_data.columns[i+1] for i in neutral])

4 pros: ['peak-entering-passengers/h', 'off-peak-entering-passengers/h', 'off-peak-passing-passengers/h', 'Station degradation level ([0,20]  scale)']
3 cons: ['peak-passing-passengers/h', 'strategic priority [0,10]', 'connectivity index [0,100]']
0 neutral: []


On a plus de pros que de cons, on peut commencer par essayer une explication 1-1 et si celle-ci ne marche pas on pourra utiliser une explication m-1 pour que 2pros puissent s'additionner pour compenser un con. 

In [60]:
result_nation_lamotte, expl_nation_lamotte = find_explanation_1_1(nation, lamotte, weights, ratp_data.columns[1:-2], 'Nation', 'La Motte-Picquet')


=== Analyse Nation > La Motte-Picquet ===
pros(Nation,La Motte-Picquet):   ['peak-entering-passengers/h', 'off-peak-entering-passengers/h', 'off-peak-passing-passengers/h', 'Station degradation level ([0,20]  scale)']
cons(Nation,La Motte-Picquet):   ['peak-passing-passengers/h', 'strategic priority [0,10]', 'connectivity index [0,100]']
Contributions: [42.99688272600236 -75.24454477050413 338.60045146726867 96.7429861335053
 -177.36214124475975 134.36525851875737 -322.4766204450177]
trade-offs valides(Nation,La Motte-Picquet): [('off-peak-entering-passengers/h', 'peak-passing-passengers/h', 263.3559066967645), ('off-peak-entering-passengers/h', 'strategic priority [0,10]', 161.23831022250891), ('off-peak-entering-passengers/h', 'connectivity index [0,100]', 16.123831022250954), ('off-peak-passing-passengers/h', 'peak-passing-passengers/h', 21.498441363001163), ('Station degradation level ([0,20]  scale)', 'peak-passing-passengers/h', 59.12071374825324)]
Pas de solution (certificat de

In [61]:
result_nation_lamotte, expl_nation_lamotte = find_explanation_m_1(nation, lamotte, weights, ratp_data.columns[1:-2], 'Nation', 'La Motte-Picquet')


=== Analyse Nation > La Motte-Picquet (type m-1) ===
pros(Nation,La Motte-Picquet):   ['peak-entering-passengers/h', 'off-peak-entering-passengers/h', 'off-peak-passing-passengers/h', 'Station degradation level ([0,20]  scale)'] contributions: [42.99688272600236, 338.60045146726867, 96.7429861335053, 134.36525851875737]
cons(Nation,La Motte-Picquet):   ['peak-passing-passengers/h', 'strategic priority [0,10]', 'connectivity index [0,100]'] contributions: [-75.24454477050413, -177.36214124475975, -322.4766204450177]

 Explication (m-1) trouvée, longueur = 3
  Trade-off: ['off-peak-passing-passengers/h'] vs (peak-passing-passengers/h)
    Contributions: [96.7429861335053] + -75.2 = 21.5
  Trade-off: ['peak-entering-passengers/h', 'Station degradation level ([0,20]  scale)'] vs (strategic priority [0,10])
    Contributions: [42.99688272600236, 134.36525851875737] + -177.4 = -0.0
  Trade-off: ['off-peak-entering-passengers/h'] vs (connectivity index [0,100])
    Contributions: [338.600451

En effet, on a ``peak-entering-passengers/h``, ``Station degradation level ([0,20]  scale)`` qui compensent ``strategic priority [0,10]``

### On montre La Motte Picquet-Grenelle > Porte d'Orléns

In [62]:
lamotte = ratp_data.iloc[4].values.flatten()[1:-2].astype(float)
orleans = ratp_data.iloc[5].values.flatten()[1:-2].astype(float)

pros = [i for i in range(len(lamotte)) if lamotte[i] > orleans[i]]
cons = [i for i in range(len(lamotte)) if lamotte[i] < orleans[i]]
neutral = [i for i in range(len(lamotte)) if lamotte[i] == orleans[i]]

print(f"{len(pros)} pros:", [ratp_data.columns[i+1] for i in pros])
print(f"{len(cons)} cons:", [ratp_data.columns[i+1] for i in cons])
print(f"{len(neutral)} neutral:", [ratp_data.columns[i+1] for i in neutral])

4 pros: ['peak-entering-passengers/h', 'peak-passing-passengers/h', 'off-peak-entering-passengers/h', 'strategic priority [0,10]']
2 cons: ['off-peak-passing-passengers/h', 'Station degradation level ([0,20]  scale)']
1 neutral: ['connectivity index [0,100]']


Pour que leurs scores soient aussi proches mais avec 4 pros contre 2 cons, les cons doivent être très grand. On va donc directement utiliser l'explication (m-1).

In [63]:
result_lamotte_orleans, expl_lamotte_orleans = find_explanation_m_1(lamotte, orleans, weights, ratp_data.columns[1:-2], 'La Motte-Picquet', 'Orléans')


=== Analyse La Motte-Picquet > Orléans (type m-1) ===
pros(La Motte-Picquet,Orléans):   ['peak-entering-passengers/h', 'peak-passing-passengers/h', 'off-peak-entering-passengers/h', 'strategic priority [0,10]'] contributions: [21.49844136300118, 37.62227238525207, 56.433408577878104, 193.48597226701065]
cons(La Motte-Picquet,Orléans):   ['off-peak-passing-passengers/h', 'Station degradation level ([0,20]  scale)'] contributions: [-112.8668171557562, -174.67483607438456]

 Explication (m-1) trouvée, longueur = 2
  Trade-off: ['peak-entering-passengers/h', 'peak-passing-passengers/h', 'off-peak-entering-passengers/h'] vs (off-peak-passing-passengers/h)
    Contributions: [21.49844136300118, 37.62227238525207, 56.433408577878104] + -112.9 = 2.7
  Trade-off: ['strategic priority [0,10]'] vs (Station degradation level ([0,20]  scale))
    Contributions: [193.48597226701065] + -174.7 = 18.8


Notre intuition était la bonne, il a fallu 3 pros pour compenser ``off-peak-passing-passengers/h``. 